In [6]:
# IMPORTS

import platform
import sys
import time
from shutil import copyfile
from tempfile import TemporaryDirectory

import numpy
from sklearn.datasets import load_breast_cancer

from concrete.ml.deployment import FHEModelClient, FHEModelDev, FHEModelServer
from concrete.ml.sklearn import XGBClassifier

In [7]:
# ENVIROMENT CLASS

class OnDiskenviroment:
    def __init__(self):
        self.server_dir = TemporaryDirectory()  
        self.client_dir = TemporaryDirectory()
        self.dev_dir = TemporaryDirectory()

    def client_send_evaluation_key_to_server(self, serialized_evaluation_keys):
        with open(self.server_dir.name + "/serialized_evaluation_keys.ekl", "wb") as f:
            f.write(serialized_evaluation_keys)

    def client_send_input_to_server_for_prediction(self, input_after_encryption):
        with open(self.server_dir.name + "/serialized_evaluation_keys.ekl", "rb") as f:
            serialized_evaluation_keys = f.read()
        time_begin = time.time()
        encrypted_prediction = FHEModelServer(self.server_dir.name).run(
            input_after_encryption, serialized_evaluation_keys
        )
        time_end = time.time()
        with open(self.server_dir.name + "/encrypted_prediction.enc", "wb") as f:
            f.write(encrypted_prediction)
        return time_end - time_begin

    def dev_send_model_to_server(self):
        copyfile(self.dev_dir.name + "/server.zip", self.server_dir.name + "/server.zip")

    def server_send_encrypted_prediction_to_client(self):
        with open(self.server_dir.name + "/encrypted_prediction.enc", "rb") as f:
            encrypted_prediction = f.read()
        return encrypted_prediction

    def dev_send_clientspecs_and_modelspecs_to_client(self):
        copyfile(self.dev_dir.name + "/client.zip", self.client_dir.name + "/client.zip")
        copyfile(
            self.dev_dir.name + "/serialized_processing.json",
            self.client_dir.name + "/serialized_processing.json",
        )

    def cleanup(self):
        self.server_dir.cleanup()
        self.client_dir.cleanup()
        self.dev_dir.cleanup()

In [8]:
# MODEL PREPARATION TO FHE AND ENVIROMENT

X, y = load_breast_cancer(return_X_y=True)

X_model_owner, X_client = X[:-500], X[-500:]
y_model_owner, y_client = y[:-500], y[-500:]

n_estimators = 10

model_to_fhe = XGBClassifier(n_bits=2, n_estimators=n_estimators, max_depth=3)
model_to_fhe.fit(X_model_owner, y_model_owner)
model_to_fhe.compile(X_model_owner)

enviroment = OnDiskenviroment()

fhemodel_to_fhe = FHEModelDev(enviroment.dev_dir.name, model_to_fhe)
fhemodel_to_fhe.save()

In [9]:
# SENDING APPROPRIATE DATA TO ENVIROMENT 

!ls -lh $enviroment.dev_dir.name
enviroment.dev_send_model_to_server()
!ls -lh $enviroment.server_dir.name
enviroment.dev_send_clientspecs_and_modelspecs_to_client()
!ls -lh $enviroment.client_dir.name

total 32K
-rw-rw-r-- 1 cybant cybant  464 lis 15 08:55 client.zip
-rw-rw-r-- 1 cybant cybant  13K lis 15 08:55 serialized_processing.json
-rw-rw-r-- 1 cybant cybant 8,5K lis 15 08:55 server.zip
total 12K
-rw-rw-r-- 1 cybant cybant 8,5K lis 15 08:55 server.zip
total 20K
-rw-rw-r-- 1 cybant cybant 464 lis 15 08:55 client.zip
-rw-rw-r-- 1 cybant cybant 13K lis 15 08:55 serialized_processing.json


In [10]:
# CLIENT PREPERATION - KEYS

client_model = FHEModelClient(enviroment.client_dir.name, key_dir=enviroment.client_dir.name)
client_model.generate_private_and_evaluation_keys()

serialized_evaluation_keys = client_model.get_serialized_evaluation_keys()

print(f"Evaluation keys size: {sys.getsizeof(serialized_evaluation_keys) / 1024 / 1024:.2f} MB")
enviroment.client_send_evaluation_key_to_server(serialized_evaluation_keys)

Evaluation keys size: 119.88 MB


KeySetCache: miss, regenerating /tmp/tmpmjmlndo0/7071120041192343400/0_0


In [11]:
# MAIN LOOP WHICH ENRYPT DATA AND MAKE STATISTICS

decrypted_predictions = []
time_exec = []
for i in range(X_client.shape[0]):
    print(i)
    buffor = X_client[[i], :]
    input_after_encryption = client_model.quantize_encrypt_serialize(buffor)
    time_exec += [enviroment.client_send_input_to_server_for_prediction(input_after_encryption)]
    encrypted_prediction = enviroment.server_send_encrypted_prediction_to_client()
    decrypted_prediction = client_model.deserialize_decrypt_dequantize(encrypted_prediction)[0]
    decrypted_predictions.append(decrypted_prediction)

clear_prediction_classes = model_to_fhe.predict_proba(X_client).argmax(axis=1)
decrypted_predictions_classes = numpy.array(decrypted_predictions).argmax(axis=1)
accuracy = (clear_prediction_classes == decrypted_predictions_classes).mean()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [12]:
# INTERESTING PRINTOUTS
print(f"Encrypted data is "f"{sys.getsizeof(input_after_encryption)/sys.getsizeof(buffor):.2f}"" larger than data")
print(f"Execution time are {[numpy.round(e, 2) for e in time_exec]}, ie an average of " f"{numpy.mean(time_exec):.2f} seconds")
print(f"Accuracy between FHE prediction and clear model is: {accuracy*100:.0f}%")

Encrypted data is 1336.48 larger than data
Execution time are [3.87, 2.91, 2.98, 2.98, 2.89, 3.07, 3.42, 3.35, 3.52, 3.35, 3.3, 3.33, 3.35, 3.38, 3.43, 3.33, 3.41, 3.5, 3.4, 3.29, 3.4, 3.38, 3.33, 3.32, 3.29, 3.35, 3.36, 3.54, 3.38, 3.38, 3.35, 3.37, 3.39, 3.39, 3.35, 3.39, 3.31, 3.43, 3.3, 3.35, 3.32, 3.42, 3.39, 3.29, 3.41, 3.41, 3.32, 3.37, 3.42, 3.42, 3.32, 3.43, 3.32, 3.26, 3.4, 3.35, 3.43, 3.39, 3.35, 3.38, 3.4, 3.44, 3.38, 3.51, 3.37, 3.54, 3.36, 3.33, 3.28, 3.39, 3.34, 3.56, 3.35, 3.39, 3.37, 3.37, 3.45, 3.42, 3.46, 3.9, 3.51, 3.28, 3.28, 3.26, 3.28, 3.4, 3.31, 3.27, 3.45, 3.37, 3.31, 3.5, 3.34, 3.25, 3.33, 3.24, 3.33, 3.31, 3.33, 3.48, 3.28, 3.28, 3.19, 3.35, 3.24, 3.26, 3.34, 3.24, 3.29, 3.32, 3.41, 3.26, 3.24, 3.34, 3.47, 3.34, 3.26, 3.31, 3.41, 3.28, 3.3, 3.28, 3.34, 3.32, 3.29, 3.36, 3.38, 3.37, 3.33, 3.26, 3.31, 3.23, 3.33, 3.28, 3.33, 3.33, 3.26, 3.35, 3.29, 3.3, 3.33, 3.29, 3.39, 3.26, 3.27, 3.35, 3.3, 3.34, 3.32, 3.28, 3.33, 3.34, 3.3, 3.33, 3.37, 3.3, 3.38, 3.26, 3.27